In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Part 1: Data Exploration and Feature Engineering

In [ ]:
#1. Preparation:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
transactions = pd.read_csv('/content/drive/MyDrive/transactions_train.csv')
customers = pd.read_csv('customers.csv')
articles = pd.read_csv('articles.csv')

# Preview the datasets
print("Transactions Preview:")
print(transactions.head())
print("Customers Preview:")
print(customers.head())
print("Articles Preview:")
print(articles.head())

#2. Data Cleaning:
#Parse date columns properly
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

#Check for Missing or Duplicated Values:
print(transactions.isnull().sum())
print(customers.isnull().sum())
print(articles.isnull().sum())
# Handle missing values
transactions.dropna(inplace=True)
customers.fillna(method='ffill', inplace=True)
articles.drop_duplicates(inplace=True)

# Merge Datasets to Create a Customer-Product View:
transactions = transactions[['customer_id', 'article_id', 't_dat']]
customers = customers[['customer_id', 'age']]
articles = articles[['article_id', 'product_type_name', 'product_code','perceived_colour_master_name','index_group_name']]

# Merge từng bước, dùng 'inner' để giảm kích thước
merged_df = transactions.merge(customers, on='customer_id', how='inner')
merged_df = merged_df.merge(articles, on='article_id', how='inner')

print(merged_df.head())


Transactions Preview:
        t_dat                                        customer_id  article_id  \
0  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   663713001   
1  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   541518023   
2  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   505221004   
3  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687003   
4  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687004   

      price  sales_channel_id  
0  0.050831                 2  
1  0.030492                 2  
2  0.015237                 2  
3  0.016932                 2  
4  0.016932                 2  
Customers Preview:
                                         customer_id   FN  Active  \
0  00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...  NaN     NaN   
1  0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...  NaN     NaN   
2  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...  NaN     NaN   
3  00005ca

<ipython-input-2-f31a31625c4b>:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  customers.fillna(method='ffill', inplace=True)


                                         customer_id  article_id      t_dat  \
0  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   663713001 2018-09-20   
1  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   541518023 2018-09-20   
2  00007d2de826758b65a93dd24ce629ed66842531df6699...   505221004 2018-09-20   
3  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687003 2018-09-20   
4  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687004 2018-09-20   

    age product_type_name  product_code perceived_colour_master_name  \
0  24.0    Underwear body        663713                        Black   
1  24.0               Bra        541518                         Pink   
2  32.0           Sweater        505221                         Pink   
3  32.0           Sweater        685687                         Pink   
4  32.0           Sweater        685687                        Green   

  index_group_name  
0       Ladieswear  
1       Ladieswear  
2          Divided  
3       

In [ ]:
# 1. Tính số lượt mua của mỗi article
top_articles = transactions['article_id'].value_counts().reset_index()
top_articles.columns = ['article_id', 'purchase_count']

# 2. Lấy 5 sản phẩm được mua nhiều nhất
top_10_articles = top_articles.head(10)

# 3. Merge với bảng articles để lấy thêm thông tin
top_10_articles_info = top_10_articles.merge(
    articles[['article_id', 'product_type_name', 'perceived_colour_master_name', 'index_group_name']],
    on='article_id',
    how='left'
)

# 4. Hiển thị kết quả
print(top_10_articles_info)


   article_id  purchase_count product_type_name perceived_colour_master_name  \
0   706016001           50287          Trousers                        Black   
1   706016002           35043          Trousers                         Blue   
2   372860001           31718             Socks                        Black   
3   610776002           30199           T-shirt                        Black   
4   759871002           26329          Vest top                        Black   
5   464297007           25025  Underwear bottom                        Black   
6   372860002           24458             Socks                        White   
7   610776001           22451           T-shirt                        White   
8   399223001           22236          Trousers                        Black   
9   706016003           21241          Trousers                         Blue   

  index_group_name  
0          Divided  
1          Divided  
2       Ladieswear  
3       Ladieswear  
4          Div

In [ ]:
bins = [0, 20, 40, 60, np.inf]  # các điểm cắt nhóm tuổi
labels = ['0-20', '21-40', '41-60', '61+']  # nhãn cho từng nhóm

customers['age_group'] = pd.cut(customers['age'], bins=bins, labels=labels, right=True)

age_group_counts = customers['age_group'].value_counts().sort_index()

print(age_group_counts)

age_group
0-20      98052
21-40    559236
41-60    329961
61+       61326
Name: count, dtype: int64
